In [ ]:
import numpy as np

sigma = 1.0 # Gaussian width
a_values = [3*sigma, 4*sigma, 6*sigma, 8*sigma] # interval half-widths to test
N_values = [512, 1024, 2048, 4096] # grid sizes to test

# Helper function for center-shifting arrays
def fftshift_like(x):
    n = x.size
    return np.concatenate([x[n//2:], x[:n//2]])

# Main loop 
for a in a_values:
    for N in N_values:

        # Define spatial grid
        dx = 2 * a / N
        x = -a + np.arange(N) * dx

        # Define Gaussian samples
        f = np.exp(-x**2 / (2 * sigma**2))

        # Construct the brute-force DFT matrix
        j = np.arange(N)
        k = np.arange(N)
        W = np.exp(-1j * 2*np.pi * np.outer(j, k) / N)  # NxN complex exponentials

        # Compute forward DFT
        fhat = W.T @ f

        # Defining the frequency grid
        q_un_shifted = fftshift_like((k - (N // 2)) * (2 * np.pi / (N * dx)))
        
        # Calculating F_num using q_un_shifted for phase factor
        F_num = dx * np.exp(1j * q_un_shifted * a) * fhat

        # Analytical Fourier Transform
        F_anal = np.sqrt(2 * np.pi) * sigma * np.exp(-0.5 * (sigma**2) * q_un_shifted**2)

        # Relative L2 error between numerical and analytical F(q)
        rel_err = np.linalg.norm(F_num - F_anal) / np.linalg.norm(F_anal)
        
        # Inverse DFT check: reconstruct f(x) from fhat
        Winv = np.exp(1j * 2*np.pi * np.outer(j, k) / N)
        f_rec = (Winv @ fhat) / N
        rec_err = np.linalg.norm(f_rec - f) / np.linalg.norm(f)

        # Print results
        print(f"a = {a:4.1f}, N = {N:5d}  |  "
              f"Relative error F(q): {rel_err:8.2e}  |  "
              f"Reconstruction error f(x): {rec_err:8.2e}")

a =  3.0, N =   512  |  Relative error F(q): 4.70e-03  |  Reconstruction error f(x): 5.94e-14
a =  3.0, N =  1024  |  Relative error F(q): 4.70e-03  |  Reconstruction error f(x): 1.24e-13
a =  3.0, N =  2048  |  Relative error F(q): 4.70e-03  |  Reconstruction error f(x): 2.62e-13
a =  3.0, N =  4096  |  Relative error F(q): 4.70e-03  |  Reconstruction error f(x): 5.19e-13
a =  4.0, N =   512  |  Relative error F(q): 1.24e-04  |  Reconstruction error f(x): 6.30e-14
a =  4.0, N =  1024  |  Relative error F(q): 1.24e-04  |  Reconstruction error f(x): 1.33e-13
a =  4.0, N =  2048  |  Relative error F(q): 1.24e-04  |  Reconstruction error f(x): 2.73e-13
a =  4.0, N =  4096  |  Relative error F(q): 1.24e-04  |  Reconstruction error f(x): 5.47e-13
a =  6.0, N =   512  |  Relative error F(q): 4.65e-09  |  Reconstruction error f(x): 6.71e-14
a =  6.0, N =  1024  |  Relative error F(q): 4.64e-09  |  Reconstruction error f(x): 1.42e-13
a =  6.0, N =  2048  |  Relative error F(q): 4.64e-09  |  Re

**(b) Effect of interval bound a:**:

For a fixed width $\sigma$, the truncation of $f(x)=e^{-x^2/(2\sigma^2)}$ to $[-a,a]$ introduces windowing errors.  
A small $a$ cuts the Gaussian tails, producing oscillations in $F(q)$; increasing $a$ reduces this effect.  
After correcting a global phase shift ($F_{\text{num}}\approx iF_{\text{anal}}$), the agreement improves for $a\gtrsim5\sigma$.

$$
a\uparrow \;\Rightarrow\; \text{smaller truncation error, better match with } F_{\text{anal}}(q).
$$

---

**(c) Effect of grid size N:**

At fixed $a$, 
$$
\Delta x = \frac{2a}{N}, \quad q_{\max}=\frac{\pi}{\Delta x}, \quad \Delta q = \frac{2\pi}{N\Delta x}.
$$
Larger $N$ gives finer $\Delta x$ and $\Delta q$, improving frequency resolution.  
After phase correction, increasing $N$ decreases the numerical error.

$$
N\uparrow \;\Rightarrow\; \text{higher resolution, smaller discretization error.}
$$

---

**(d) Inverse DFT check:**

The numerical inverse 
$$
f_j^{(\text{rec})} = \frac{1}{N}\sum_{k=0}^{N-1}\hat f_k e^{i2\pi jk/N}
$$
reproduces the original samples with relative error $\sim10^{-13}$,  
confirming the correctness of the DFT/IDFT pair.
